In [1]:
include("../encoding.jl")

find_normalization (generic function with 1 method)

In [2]:
using LinearAlgebra

In [3]:
function printresults(x)
    Δx = encode(encoder, x)
    ϕx = encode(encoder, x, normalized=false);
    fx = f.(x);

    # 1. [S+] - [S_-]
    Sp = sum(ϕx[fx.>0,:], dims=1)[:];
    Sn = sum(ϕx[fx.<0,:], dims=1)[:];
    pred = ((sign.(Sp) - sign.(Sn))'*ϕx')[:];
    acc = Statistics.mean(sign.(pred).==fx)
    println(acc)

    # 2. [S+-S_-]
    pred = ((sign.(Sp .- Sn))'*ϕx')[:];
    acc = Statistics.mean(sign.(pred).==fx)
    println(acc)

    # 3. S+ - S-
    pred = ((Sp .- Sn)'*ϕx')[:];
    acc = Statistics.mean(sign.(pred).==fx)
    println(acc)

    # 4. F
    P = mean(Δx, dims=1)
    px = P*Δx' /D
    F = Δx'*(fx ./ px[:])
    pred= ((Δx*F)[:]/ D);
    acc = Statistics.mean(sign.(pred).==fx)
    println(acc)

    # 5. [F]
    pred= ((Δx*sign.(F))[:]/ D);
    acc = Statistics.mean(sign.(pred).==fx)
    println(acc)

    # 6. P
    Vpos = rand([-1,1],D)
    Vneg = rand([-1,1],D)
    P = Vpos.* sum(Δx[fx.>0,:], dims=1)[:] .+ Vneg .* sum(Δx[fx.<0,:], dims=1)[:] #sum! do not use mean! (messes with the prior)
    pred = (Δx .* P') * (Vpos .- Vneg)
    acc = Statistics.mean(sign.(pred).==fx)
    println(acc)

    # 7. [P]
    pred = (Δx .* sign.(P)') * (Vpos .- Vneg)
    acc = Statistics.mean(sign.(pred).==fx)
    println(acc)

    #8. M - λ=1
    function solve_ridge(X::AbstractMatrix,B;λ=1)
        F =  inv(X'*X + λ*I)*X'*B
        return F
    end
    function solve_ridge_woodbury(X::AbstractMatrix,B;λ=1)
        F = (I/λ - X'*inv(I+X*X'/λ)*X /λ^2)*X'*B
        return F
    end
    M = solve_ridge_woodbury(Δx, fx, λ=1) #very few data points while many dimensions: efficient woodbury
    pred= ((Δx*M)[:]/ D);
    acc = Statistics.mean(sign.(pred).==fx)
    println(acc)

    #9. [M] - λ=1
    pred= ((Δx*sign.(M))[:]/ D);
    acc = Statistics.mean(sign.(pred).==fx)
    println(acc, " bipolar approximation [M] if ridge parameter to small: less robust for approximating")

    #8. M - λ=1e4
    M = solve_ridge_woodbury(Δx, fx, λ=1e4) #very few data points while many dimensions: efficient woodbury
    pred= ((Δx*M)[:]/ D);
    acc = Statistics.mean(sign.(pred).==fx)
    println(acc)

    #9. [M] - λ=1e4
    pred= ((Δx*sign.(M))[:]/ D);
    acc = Statistics.mean(sign.(pred).==fx)
    println(acc)
end

printresults (generic function with 1 method)

In [4]:
l, D, bipolar = 0.1, 20_000, true
encoder = realvalue_encoder(l,D, bipolar=bipolar);

In [5]:
a,b = 0.145651, 0.3565
function f(x)
    if x> a && x<b
        return 1.
    else
        return -1.
    end
end

f (generic function with 1 method)

In [6]:
x = range(0,1,length=100)
printresults(x)

0.9
0.98
1.0
1.0
1.0
1.0
1.0
1.0
0.79 bipolar approximation [M] if ridge parameter to small: less robust for approximating
1.0
1.0


In [7]:
x = vcat(collect(range(0,a,length=26)[1:end-1]), collect(range(a,b,length=50)), collect(range(b,1,length=26)[2:end]))
printresults(x)

0.95
0.96
0.97
0.98
0.98
0.97
0.96
1.0
0.52 bipolar approximation [M] if ridge parameter to small: less robust for approximating
1.0
1.0
